In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

2024-11-04 13:54:35.719846: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-04 13:54:35.863060: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 13:54:38.945641: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 13:54:40.711384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730724882.449223 3491538 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730724882.88

In [3]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
# Setting these as global variables so that they can be shuffled to ensure an even distribution- many times I wasn't getting all 24/720 labels in the validation set
global imgs, train_imgs, val_imgs, test_imgs, train_labels, val_labels, test_labels, distributed

images = np.load('75/images.npy')
images = images.astype('float32')

# Function that shuffles the images and labels into training, validation, and test sets
def initalize():
    indices = np.random.permutation(images.shape[0])
    imgs = images[indices]

    split_1 = int(18000*0.8)
    split_2 = int(18000*0.9)

    train_imgs = imgs[:split_1]
    val_imgs = imgs[split_1:split_2]
    test_imgs = imgs[split_2:]

    # Normalizing the images
    train_imgs = train_imgs / 255.0
    test_imgs = test_imgs / 255.0
    val_imgs = val_imgs / 255.0

    labels = np.load('75/labels.npy')
    labels = labels.astype('int32')
    labels = labels[indices]
    train_labels = labels[:split_1]
    val_labels = labels[split_1:split_2]
    test_labels = labels[split_2:]

    train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
    val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
    test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))


## Classification Model

In [ ]:
# Convert the time into 24 separate labels
def conv_time_24(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

# Convert the time into 720 separate labels 
def conv_time_720(time):
    return time[0]*60 + time[1]

conv_time = conv_time_24
while True:
    initalize()
    train_labels_converted = np.array([conv_time(time) for time in train_labels])
    test_labels_converted = np.array([conv_time(time) for time in test_labels])
    val_labels_converted = np.array([conv_time(time) for time in val_labels])

    encoder = LabelEncoder()
    test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
    train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
    val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

    OHencoder = OneHotEncoder(sparse_output=False)
    train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
    val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))

    # Check if all labels are present in the validation set, if not, reshuffle
    try:
        val_labels_oh = val_labels_oh.reshape((val_labels_oh.shape[0], 24))
        break
    except:
        pass

In [ ]:
input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(5,5), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))

model.add(keras.layers.Flatten())
# First layer - one neuron for each pixel
model.add(keras.layers.Dense(units=5625, activation="relu"))
model.add(keras.layers.Dense(units=1125, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(keras.layers.Dense(units=24, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

print(train_imgs.shape, train_labels_oh.shape)


model.fit(train_imgs, train_labels_oh, epochs=10, batch_size=256, validation_data=(val_imgs, val_labels_oh), callbacks=[early_stop])

(14400, 75, 75, 1) (14400, 24)
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.0423 - loss: 3.1781 - val_accuracy: 0.0361 - val_loss: 3.1782
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.0428 - loss: 3.1779 - val_accuracy: 0.0356 - val_loss: 3.1784
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.0435 - loss: 3.1778 - val_accuracy: 0.0361 - val_loss: 3.1784
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.0442 - loss: 3.1779 - val_accuracy: 0.0361 - val_loss: 3.1787
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.0418 - loss: 3.1778 - val_accuracy: 0.0361 - val_loss: 3.1789
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.0432 - loss: 3.1776 - val_accuracy: 0.0361 - val_loss: 3.1788
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.0430 - loss: 3.1776 - val_accuracy: 0.0361 - val_loss: 3.1790
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.0424 - loss: 3.

In [54]:
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)

results = encoder.inverse_transform(preds)
accuracy = np.sum(results == test_labels_converted) / len(test_labels_converted)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
5.111111111111112 %


## Regression Model

In [51]:
import tensorflow.keras.backend as K

#returning the common sense difference between two times
def difference_func(true, pred):
    true = K.cast(true, 'float32')
    diff_1 = K.abs(true - pred)
    diff_2 = K.abs(true - (pred + 12*60))

    return K.minimum(diff_1, diff_2)

In [ ]:
initalize()

def conv_time(time):
    return round(time[0] + time[1]/60, 3)

train_labels_reg = np.array([conv_time(time) for time in train_labels])
test_labels_reg = np.array([conv_time(time) for time in test_labels])
val_labels_reg = np.array([conv_time(time) for time in val_labels])

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), strides=(2,2),input_shape=(75, 75, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=1, activation="softplus"))
model.compile(loss=difference_func, optimizer="adam", metrics=[difference_func])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, train_labels_reg, epochs=40, batch_size = 512, validation_data = (val_imgs, val_labels_reg), callbacks = [early_stop])

Epoch 1/40


/vol/home/s4201388/Documents/Leiden/IDL/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - difference_func: 4.0812 - loss: 4.0826 - val_difference_func: 3.0844 - val_loss: 3.0844
Epoch 2/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - difference_func: 3.0233 - loss: 3.0273 - val_difference_func: 3.0645 - val_loss: 3.0648
Epoch 3/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - difference_func: 3.0027 - loss: 3.0040 - val_difference_func: 3.0845 - val_loss: 3.0849
Epoch 4/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - difference_func: 3.0454 - loss: 3.0440 - val_difference_func: 3.0875 - val_loss: 3.0880
Epoch 5/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - difference_func: 3.0463 - loss: 3.0454 - val_difference_func: 3.1041 - val_loss: 3.1049
Epoch 6/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - difference_func: 2.9913 - loss: 2.9910 - val_difference_func: 3.2082 - val_loss: 3.2088
Epoch 7/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - difference_func: 3.0908 - loss: 3.0882 - val_difference_func: 3.0730 - val_loss: 3.0736
Epoch 8/40
29/29 ━━━━━━━

In [66]:
reg_preds = model.predict(test_imgs)
for i in range(len(reg_preds)):
    print(reg_preds[i], test_labels_reg[i])
accuracy = np.mean(abs(reg_preds - test_labels_reg) < 0.01)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[5.71604] 8.033
[5.6474957] 6.817
[5.639689] 10.383
[5.6338997] 9.133
[5.6161847] 5.233
[5.702291] 5.017
[5.6282105] 2.45
[5.587209] 4.017
[5.6159406] 6.917
[5.7096915] 11.133
[5.6929803] 1.6
[5.646908] 8.067
[5.61418] 11.1
[5.6245203] 3.05
[5.650396] 3.533
[5.6066213] 0.167
[5.6142445] 0.167
[5.7441306] 1.783
[5.6395707] 0.2
[5.6673965] 6.65
[5.6935906] 7.383
[5.68261] 6.633
[5.707229] 0.783
[5.7158217] 10.983
[5.623289] 5.017
[5.6584697] 2.933
[5.7342534] 4.483
[5.560532] 1.033
[5.6740646] 8.5
[5.671574] 7.267
[5.661425] 2.217
[5.6307955] 7.45
[5.6232233] 9.883
[5.6495075] 4.167
[5.6040087] 9.483
[5.6861367] 1.833
[5.632621] 2.467
[5.611585] 4.517
[5.69044] 8.333
[5.6951957] 7.25
[5.7013354] 3.25
[5.6980634] 0.083
[5.6691236] 4.45
[5.638679] 6.733
[5.6636944] 10.833
[5.680578] 4.317
[5.706926] 6.567
[5.6353083] 7.183
[5.5832253] 0.933
[5.617577] 1.667
[5.6277084] 6.783
[5.684453] 9.1
[5.631188] 7.033
[5.6630063] 5.333
[5.623227] 7.583
[5.5570865

## Two-Headed Model

In [26]:
train_hours = train_labels[:, 0]
train_minutes = train_labels[:, 1]

val_hours = val_labels[:, 0]
val_minutes = val_labels[:, 1]

test_hours = test_labels[:, 0]
test_minutes = test_labels[:, 1]

In [27]:
inp = keras.layers.Input(shape = (75,75,1))
model = keras.layers.Convolution2D(32,kernel_size = (5,5), strides= (2,2), activation = "relu")(inp)
model = keras.layers.MaxPooling2D(pool_size =2)(model)
model = keras.layers.Convolution2D(32,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.Convolution2D(32,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.MaxPooling2D(pool_size =2)(model)
model = keras.layers.Convolution2D(64,kernel_size = (3,3),activation = "relu")(model)
model = keras.layers.Convolution2D(64,kernel_size = (1,1),activation = "relu")(model)
model = keras.layers.Flatten()(model)

d = keras.layers.Dense(256,activation = "relu")(model)
d = keras.layers.Dense(256,activation = "relu")(d)
d = keras.layers.Dropout(0.1)(d)
d = keras.layers.Dense(256,activation = "relu")(d)

hour = keras.layers.Dense(256,activation = "relu")(d)
hour = keras.layers.Dense(128,activation = "relu")(hour)
hour = keras.layers.Dense(64,activation = "relu")(hour)
hour = keras.layers.Dense(32,activation = "relu")(hour)
hour = keras.layers.Dense(16,activation = "relu")(hour)
hour = keras.layers.Dense(12,activation= "softmax", name= "hour")(hour)

minute = keras.layers.Dense(256,activation = "relu")(d)
minute = keras.layers.Dense(256,activation = "relu")(minute)
minute = keras.layers.Dense(256,activation = "relu")(minute)
minute = keras.layers.Dense(128,activation = "relu")(minute)
minute = keras.layers.Dense(64,activation = "relu")(minute)
minute = keras.layers.Dense(32,activation = "relu")(minute)
minute = keras.layers.Dense(16,activation = "relu")(minute)
minute = keras.layers.Dense(1, activation = "softplus", name = "minute")(minute)

model = tf.keras.models.Model(inputs=inp, outputs=[hour, minute])
optim = tf.keras.optimizers.Adam()
model.compile(loss=['sparse_categorical_crossentropy', 'mse'], optimizer=optim, metrics=['accuracy',"mae"])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, [train_hours, train_minutes], epochs=30, batch_size = 512, validation_data = (val_imgs, [val_hours, val_minutes]), callbacks = [early_stop])

Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - hour_accuracy: 0.0808 - hour_loss: 2.4887 - loss: 838.4274 - minute_loss: 835.5597 - minute_mae: 23.9306 - val_hour_accuracy: 0.0839 - val_hour_loss: 2.4879 - val_loss: 318.8836 - val_minute_loss: 316.8258 - val_minute_mae: 15.3285
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - hour_accuracy: 0.0845 - hour_loss: 2.4889 - loss: 322.3623 - minute_loss: 319.8416 - minute_mae: 15.2535 - val_hour_accuracy: 0.0861 - val_hour_loss: 2.4912 - val_loss: 308.0358 - val_minute_loss: 305.8560 - val_minute_mae: 15.1308
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - hour_accuracy: 0.0859 - hour_loss: 2.4869 - loss: 303.3432 - minute_loss: 300.8137 - minute_mae: 14.9939 - val_hour_accuracy: 0.0661 - val_hour_loss: 2.4906 - val_loss: 303.1655 - val_minute_loss: 300.7621 - val_minute_mae: 15.0053
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - hour_accuracy: 0.0851 - hour_loss: 2.4860 - loss: 308.8290 - minute_loss: 306.3998 - minute_

In [31]:
predictions = model.predict(test_imgs)
hour_p = np.argmax(predictions[0], axis = 1)
minutes_p = predictions[1]

accuracy = np.mean(np.abs(hour_p - test_hours) < 1) * np.mean(np.abs(minutes_p - test_minutes) < 5)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1.363423353909465 %


## Label Transformation using Periodic Function 


In [34]:
sine_time_train = (train_hours*60 + train_minutes) 
sine_time_test = (test_hours*60 + test_minutes)  
sine_time_valid = (val_hours*60 + val_minutes) 

In [35]:
np.max(sine_time_train)

np.int32(719)

In [44]:
sine_angle_test = (sine_time_test/720)*2*np.pi
sine_angle_train = (sine_time_train/720)*2*np.pi
sine_angle_valid = (sine_time_valid/720)*2*np.pi

In [49]:
# Sine Regression CNN

sin_reg = keras.models.Sequential()
sin_reg.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), strides = (2,2),input_shape=(75, 75, 1)))
sin_reg.add(keras.layers.MaxPooling2D(pool_size=2))
sin_reg.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
sin_reg.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))
sin_reg.add(keras.layers.MaxPooling2D(pool_size=2))
sin_reg.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
sin_reg.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))


sin_reg.add(keras.layers.Flatten())
sin_reg.add(keras.layers.Dense(units=512, activation='relu'))
sin_reg.add(keras.layers.Dense(units=512, activation='relu'))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dropout(0.2))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dense(units=256, activation='relu'))
sin_reg.add(keras.layers.Dropout(0.2))
sin_reg.add(keras.layers.Dense(units=128, activation='relu'))
sin_reg.add(keras.layers.Dense(units=64, activation='relu'))
sin_reg.add(keras.layers.Dense(units=32, activation='relu'))
sin_reg.add(keras.layers.Dense(units=1, activation="softplus"))
early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
sin_reg.compile(loss='mse', optimizer= optimizer, metrics=['mae'])
sin_reg.fit(train_imgs, sine_angle_train, epochs=45, batch_size = 512, validation_data = (val_imgs, sine_angle_valid), callbacks = [early_stop])

Epoch 1/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 0.1435 - mae: 0.2371 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 2/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7896e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 3/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 3.7661e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 4/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7713e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 5/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7717e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 6/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 3.8013e-05 - mae: 0.0056 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 7/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7782e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoch 8/45
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 3.7938e-05 - mae: 0.0055 - val_loss: 3.8348e-05 - val_mae: 0.0056
Epoc

In [47]:
predictions = sin_reg.predict(test_imgs)

def difference_func(pred,y):
  pred = np.transpose(pred)
  diff_one = np.maximum(pred,y) - np.minimum(pred,y)
  diff_two = np.minimum(pred,y) + 1 - np.maximum(pred,y)
  return np.minimum(diff_one,diff_two)

result = difference_func(predictions,sine_angle_test).reshape(-1)

accuracy = np.mean(result < np.pi/6)
print(accuracy*100, '%')

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
100.0 %
